In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import json

In [2]:
import os

# Define the data folder path
data_folder = 'data/question_1a/'

# Get all JSON files in the folder
json_files = [f for f in os.listdir(data_folder) if f.endswith('.json')]

# Loop through all JSON files and load them into separate variables
for filename in json_files:
    filepath = os.path.join(data_folder, filename)
    # Use filename without extension as variable name
    var_name = filename.replace('.json', '')
    
    with open(filepath, 'r') as f:
        # Create a variable with the same name as the file
        globals()[var_name] = json.load(f)
    
    print(f"Loaded: {filename} -> {var_name}")

# Now you have separate dictionaries:
# appliance_params
# bus_params
# consumer_params
# DER_production
# usage_preference

print(f"\nLoaded {len(json_files)} files as separate dictionaries:")
for filename in json_files:
    var_name = filename.replace('.json', '')
    print(f"  - {var_name}")

Loaded: appliance_params.json -> appliance_params
Loaded: bus_params.json -> bus_params
Loaded: consumer_params.json -> consumer_params
Loaded: DER_production.json -> DER_production
Loaded: usage_preference.json -> usage_preference

Loaded 5 files as separate dictionaries:
  - appliance_params
  - bus_params
  - consumer_params
  - DER_production
  - usage_preference


In [3]:
# Extract key parameters from the loaded data
print("=== EXTRACTING PARAMETERS ===")

# Time horizon
T = 24  # 24 hours

# Bus parameters
bus_data = bus_params[0]
import_tariff = bus_data['import_tariff_DKK/kWh']
export_tariff = bus_data['export_tariff_DKK/kWh']
max_import = bus_data['max_import_kW']
max_export = bus_data['max_export_kW']
energy_prices = bus_data['energy_price_DKK_per_kWh']  # 24-hour prices

print(f"Import tariff: {import_tariff} DKK/kWh")
print(f"Export tariff: {export_tariff} DKK/kWh")
print(f"Max import/export: {max_import}/{max_export} kW")

# PV parameters
pv_data = appliance_params['DER'][0]
pv_max_power = pv_data['max_power_kW']
pv_profile = DER_production[0]['hourly_profile_ratio']
pv_max_hourly = [pv_max_power * ratio for ratio in pv_profile]

print(f"PV max power: {pv_max_power} kW")
print(f"PV daily production profile: {len(pv_profile)} hours")

# Load parameters
load_data = appliance_params['load'][0]
load_max_hourly = load_data['max_load_kWh_per_hour']
min_daily_consumption = usage_preference[0]['load_preferences'][0]['min_total_energy_per_day_hour_equivalent']

print(f"Load max hourly: {load_max_hourly} kWh/hour")
print(f"Min daily consumption: {min_daily_consumption} kWh")

print("\n=== ENERGY PRICES BY HOUR ===")
for t in range(T):
    print(f"Hour {t:2d}: {energy_prices[t]:5.2f} DKK/kWh, PV max: {pv_max_hourly[t]:4.2f} kW")

=== EXTRACTING PARAMETERS ===
Import tariff: 5 DKK/kWh
Export tariff: 4 DKK/kWh
Max import/export: 1000/500 kW
PV max power: 3.0 kW
PV daily production profile: 24 hours
Load max hourly: 3.0 kWh/hour
Min daily consumption: 8 kWh

=== ENERGY PRICES BY HOUR ===
Hour  0:  1.10 DKK/kWh, PV max: 0.00 kW
Hour  1:  1.05 DKK/kWh, PV max: 0.00 kW
Hour  2:  1.00 DKK/kWh, PV max: 0.00 kW
Hour  3:  0.90 DKK/kWh, PV max: 0.00 kW
Hour  4:  0.85 DKK/kWh, PV max: 0.00 kW
Hour  5:  1.01 DKK/kWh, PV max: 0.15 kW
Hour  6:  1.05 DKK/kWh, PV max: 0.42 kW
Hour  7:  1.20 DKK/kWh, PV max: 0.63 kW
Hour  8:  1.40 DKK/kWh, PV max: 0.45 kW
Hour  9:  1.60 DKK/kWh, PV max: 0.36 kW
Hour 10:  1.50 DKK/kWh, PV max: 0.63 kW
Hour 11:  1.10 DKK/kWh, PV max: 0.75 kW
Hour 12:  1.05 DKK/kWh, PV max: 2.55 kW
Hour 13:  1.00 DKK/kWh, PV max: 2.25 kW
Hour 14:  0.95 DKK/kWh, PV max: 1.65 kW
Hour 15:  1.00 DKK/kWh, PV max: 1.29 kW
Hour 16:  1.20 DKK/kWh, PV max: 0.69 kW
Hour 17:  1.50 DKK/kWh, PV max: 0.15 kW
Hour 18:  2.10 DKK/k

In [4]:
# Create Gurobi model
print("\n=== CREATING OPTIMIZATION MODEL ===")
model = gp.Model("Energy_Optimization")

# Decision Variables
print("Creating decision variables...")

# PV generation (can be curtailed)
pv_gen = model.addVars(T, lb=0, name="pv_gen")  # kW

# Flexible load consumption  
load_cons = model.addVars(T, lb=0, name="load_cons")  # kW

# Grid imports/exports
grid_import = model.addVars(T, lb=0, name="grid_import")  # kW (positive = import)
grid_export = model.addVars(T, lb=0, name="grid_export")  # kW (positive = export)

print(f"Created variables for {T} time periods")

# Constraints
print("Adding constraints...")

# 1. PV generation limit (can be curtailed)
pv_limit_cons = model.addConstrs(
    (pv_gen[t] <= pv_max_hourly[t] for t in range(T)), 
    name="pv_limit"
)

# 2. Load limit constraint
load_limit_cons = model.addConstrs(
    (load_cons[t] <= load_max_hourly for t in range(T)), 
    name="load_limit"
)

# 3. Grid import/export limits
import_limit_cons = model.addConstrs(
    (grid_import[t] <= max_import for t in range(T)), 
    name="import_limit"
)

export_limit_cons = model.addConstrs(
    (grid_export[t] <= max_export for t in range(T)), 
    name="export_limit"
)

# 4. Energy balance constraint (power balance at each hour)
balance_cons = model.addConstrs(
    (pv_gen[t] + grid_import[t] == load_cons[t] + grid_export[t] for t in range(T)), 
    name="energy_balance"
)

# 5. Minimum daily energy consumption requirement
min_consumption_cons = model.addConstr(
    gp.quicksum(load_cons[t] for t in range(T)) >= min_daily_consumption, 
    name="min_daily_consumption"
)

print(f"Added constraints:")
print(f"  - PV generation limits: {T}")
print(f"  - Load limits: {T}")  
print(f"  - Grid import limits: {T}")
print(f"  - Grid export limits: {T}")
print(f"  - Energy balance: {T}")
print(f"  - Min daily consumption: 1")

model.update()
print(f"\nModel has {model.numVars} variables and {model.numConstrs} constraints")


=== CREATING OPTIMIZATION MODEL ===
Set parameter Username
Set parameter LicenseID to value 2670718
Academic license - for non-commercial use only - expires 2026-05-26
Creating decision variables...
Created variables for 24 time periods
Adding constraints...
Added constraints:
  - PV generation limits: 24
  - Load limits: 24
  - Grid import limits: 24
  - Grid export limits: 24
  - Energy balance: 24
  - Min daily consumption: 1

Model has 96 variables and 121 constraints


In [5]:
# Objective Function: Minimize daily energy procurement cost
print("\n=== SETTING OBJECTIVE FUNCTION ===")

# Cost components:
# 1. Energy costs: (import * price - export * price) 
# 2. Grid tariffs: (import * import_tariff + export * export_tariff)

energy_cost = gp.quicksum(
    grid_import[t] * energy_prices[t] - grid_export[t] * energy_prices[t] 
    for t in range(T)
)

tariff_cost = gp.quicksum(
    grid_import[t] * import_tariff + grid_export[t] * export_tariff 
    for t in range(T)
)

total_cost = energy_cost + tariff_cost

model.setObjective(total_cost, GRB.MINIMIZE)

print("Objective: Minimize (Energy Cost + Tariff Cost)")
print("  Energy Cost = Σ(import[t] * price[t] - export[t] * price[t])")
print("  Tariff Cost = Σ(import[t] * import_tariff + export[t] * export_tariff)")

# Solve the model
print("\n=== SOLVING MODEL ===")
model.optimize()

if model.status == GRB.OPTIMAL:
    print(f"✓ Optimal solution found!")
    print(f"Optimal cost: {model.objVal:.2f} DKK")
    
    # Calculate cost breakdown
    energy_cost_val = sum(
        grid_import[t].x * energy_prices[t] - grid_export[t].x * energy_prices[t] 
        for t in range(T)
    )
    tariff_cost_val = sum(
        grid_import[t].x * import_tariff + grid_export[t].x * export_tariff 
        for t in range(T)
    )
    
    print(f"  Energy cost: {energy_cost_val:.2f} DKK")
    print(f"  Tariff cost: {tariff_cost_val:.2f} DKK")
    
    # Summary statistics
    total_pv_gen = sum(pv_gen[t].x for t in range(T))
    total_load = sum(load_cons[t].x for t in range(T))
    total_import = sum(grid_import[t].x for t in range(T))
    total_export = sum(grid_export[t].x for t in range(T))
    
    print(f"\n=== DAILY SUMMARY ===")
    print(f"Total PV generation: {total_pv_gen:.2f} kWh")
    print(f"Total load consumption: {total_load:.2f} kWh")
    print(f"Total grid import: {total_import:.2f} kWh")
    print(f"Total grid export: {total_export:.2f} kWh")
    print(f"Net grid consumption: {total_import - total_export:.2f} kWh")
    
else:
    print(f"❌ Model not optimal. Status: {model.status}")
    if model.status == GRB.INFEASIBLE:
        print("Model is infeasible!")


=== SETTING OBJECTIVE FUNCTION ===
Objective: Minimize (Energy Cost + Tariff Cost)
  Energy Cost = Σ(import[t] * price[t] - export[t] * price[t])
  Tariff Cost = Σ(import[t] * import_tariff + export[t] * export_tariff)

=== SOLVING MODEL ===
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 8840HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 121 rows, 96 columns and 216 nonzeros
Model fingerprint: 0xb443524a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-01, 1e+03]
Presolve removed 105 rows and 42 columns
Presolve time: 0.01s
Presolved: 16 rows, 54 columns, 69 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   8.000000e+00   0.000000e+00      0s
       9    0.00